In [12]:
###################################################################################################
# 正则化测试实例

import tensorflow as tf

weight = tf.constant([[1.0, -2.0], [-3.0, 4.0]])

with tf.Session() as sess:
    # (|1.0| + |-2.0| + |-3.0| + |4.0|) * 0.5 = 5
    print sess.run(tf.contrib.layers.l1_regularizer(.5)(weight))
    # (1.0^2 + (-2.0)^2 + (-3.0)^2 + (4.0)^2 * 0.5 = 7.5
    print sess.run(tf.contrib.layers.l2_regularizer(.5)(weight))

5.0
7.5


In [9]:
###################################################################################################
# 正则化测试实例
# 通过集合的方法计算5层全连接神经网络结构L2正则化的损失函数
import tensorflow as tf

x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 2))
batch_size = 8

# 定义每一层网络节点个数
layer_dimension = [2, 10, 10, 10, 1]
n_layers = len(layer_dimension)    # 神经网络的层数
cur_layer = x     # cur_layer 维护前向传播时最深层的节点，初始为输入层
in_dimension = layer_dimension[0]    # 当前层节点个数


# 获取每一层边上的权重，并将这个权重的L2正则化损失加入名称“losses”的集合中
def get_weight(shape, _lambda):
    # 生成一个shape维度的变量
    var = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    # add_to_collection 将这个新生成的变量的L2正则损失加入到集合中
    tf.add_to_collection("losses", tf.contrib.layers.l2_regularizer(_lambda)(var))
    return var


# 通过循环生成一个5层全连接的神经网络结构
for i in range(1, n_layers):
    out_dimension = layer_dimension[i]    # 下一层节点个数
    # 生成当前层中权重的变量，并将这个变量的L2正则化损失加入计算图上的集合中
    weight = get_weight([in_dimension, out_dimension], 0.001)
    bias = tf.Variable(tf.constant(0.1, shape=[out_dimension]))
    
    # 使用relu激活函数
    cur_layer = tf.nn.relu(tf.matmul(cur_layer, weight) + bias)
    # 进入下一层节点之前将下一层的节点更新为当前节点的个数
    in_dimension = out_dimension
    

# 
mse_loss = tf.reduce_mean(tf.square(y_ - cur_layer))

tf.add_to_collection("losses", mse_loss)

print tf.get_collection("losses")

# get_collection 返回一个列表，存放的是损失函数的不同部分，将其全部加起来即为最终的损失函数
loss = tf.add_n(tf.get_collection("losses"))

[<tf.Tensor 'l2_regularizer:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_1:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_2:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_3:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_4:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_5:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_6:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_7:0' shape=() dtype=float32>, <tf.Tensor 'Mean:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_8:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_9:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_10:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_11:0' shape=() dtype=float32>, <tf.Tensor 'Mean_1:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_12:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_13:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_14:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer_15:0' shape=() dtype=f

In [15]:
###################################################################################################
# 滑动平均模型使用

import tensorflow as tf

# 定义一个变量用来计算滑动平均变量，初始值设置0.
v1 = tf.Variable(0, dtype=tf.float32)

# 定义一个模拟神经网络中迭代的轮数变量，用来动态控制衰减率
step = tf.Variable(0, trainable=False)

# 定义一个滑动平均类（class）。 初始化时给定衰减率和控制衰减率的变量step
ema = tf.train.ExponentialMovingAverage(0.99, step)

# 定义一个更新滑动平均的操作。 这里需要给定一个列表，每次执行这个操作时这个列表中的变量都会更新
maintain_average_op = ema.apply([v1])

###################################################################################################
# 训练结果

with tf.Session() as sess:
    # 初始化所有变量
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    
    # 通过ema.average(v1)获取滑动平均之后变量的取值
    print "通过ema.average(v1)获取滑动平均之后变量的取值"
    print sess.run([v1, ema.average(v1)])
    
    # 更新变量v1的值到5
    print "更新变量v1的值到5"
    sess.run(tf.assign(v1, 5))
    # 更新v1的滑动平均值。 衰减率为： min{0.99, (1 + step) / (10 + step) = 0.1} = 0.1
    # 所以v1的滑动平均会更新为 0.1 * 0 + 0.9 * 5 = 4.5
    sess.run(maintain_average_op)
    print sess.run([v1, ema.average(v1)])
    
    # step=10000， v1 = 10, 更新v1的滑动平均值
    # 衰减率为： min{0.99, (1 + step) / (10 + step) = 0.999} = 0.99
    # v1 的滑动平均值更新为： 0.99 * 4.5 + 0.01 * 10 = 4.555
    print "step=10000， v1 = 10, 更新v1的滑动平均值"
    sess.run(tf.assign(step, 10000))
    sess.run(tf.assign(v1, 10))
    sess.run(maintain_average_op)
    print sess.run([v1, ema.average(v1)])
    
    print "再次更新滑动平均值"
    # 再次更新滑动平均值
    # v1 的滑动平均值更新为：0.99 * 4.5549998 + 0.01 * 10 = 4.60945
    sess.run(maintain_average_op)
    print sess.run([v1, ema.average(v1)])

通过ema.average(v1)获取滑动平均之后变量的取值
[0.0, 0.0]
更新变量v1的值到5
[5.0, 4.5]
step=10000， v1 = 10, 更新v1的滑动平均值
[10.0, 4.5549998]
再次更新滑动平均值
[10.0, 4.6094499]
